In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# 정규식
import re

# one-hot encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# 데이터 불러오기

In [2]:
df1 = pd.read_csv("./data/경주마_정보.csv", encoding ="cp949")
df1

,마번,마명,등급,성별,연령,레이팅1,전적,수득상금(천원),조교사,마주,부마명,모마명
0,45027,가나패션,국4,암,3,38,5(0/4/1),"40,800",안해양(26),김한수,올드패션드,찬란한여명
1,39021,가로수길,국2,수,6,73,13(5/3/0),"202,880",리카디(34),홍종철,채플로열,협동
2,43765,갈색향기,국5,암,3,26,10(0/2/1),"22,900",휴양마(999),김광명,메니피,휴지호프
3,44650,감곡선샤인,국5,암,3,30,6(1/0/0),"14,750",심승태(37),손칠규,에이스갤러퍼,드림갤러퍼
4,45187,강력한지금,국4,수,2,37,6(1/1/0),"27,550",홍대유(6),금악목장,페더럴리스트,금악여제
...,...,...,...,...,...,...,...,...,...,...,...,...
1308,43514,히든삭스,외2,수,5,80,24(3/5/3),"238,700",강성오(9),김창식,RUN AWAY AND HIDE,DELIGHTFUL DAWN
1309,45908,히어로사랑,국6,암,2,0,1(0/1/0),"13,200",안해양(26),임재형,위드디스팅션,지봉사랑
1310,45380,히어로퀸,국6,암,2,0,1(0/0/0),0,안해양(26),임재형,선더모카신,라이브퀸
1311,42707,히트에이스,국5,암,4,28,12(0/2/1),"19,240",이관호(42),유수현,메니피,천년나래


In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   마번        1313 non-null   int64 
 1   마명        1313 non-null   object
 2   등급        1313 non-null   object
 3   성별        1313 non-null   object
 4   연령        1313 non-null   int64 
 5   레이팅1      1313 non-null   int64 
 6   전적        1313 non-null   object
 7   수득상금(천원)  1313 non-null   object
 8   조교사       1313 non-null   object
 9   마주        1313 non-null   object
 10  부마명       1313 non-null   object
 11  모마명       1313 non-null   object
dtypes: int64(3), object(9)
memory usage: 123.2+ KB


In [4]:
# 마명, 마번은 유니크한 값인 거 확인
print(len(df1.iloc[:,1].unique()))      # 마명
print(len(df1.iloc[:,0].unique()))      # 마번

1313
1313


## 등급

In [5]:
# 등급은 국1~6, 외1~4등급 까지 있다.
df1.iloc[:,2].unique()

array(['국4', '국2', '국5', '국6', '국1', '외1', '국3', '외3', '외4', '외2'],
      dtype=object)

In [6]:
# 등급에 따른 갯수
df1.loc[:,'등급'].value_counts()

국6    362
국5    326
국4    252
국3    111
외4     68
외2     47
국2     44
외3     37
국1     33
외1     33
Name: 등급, dtype: int64

## 성별

In [7]:
# 성별 종류: 암 = 암말, 수말, 거세수말
df1.loc[:, '성별'].unique()

array(['암', '수', '거'], dtype=object)

In [8]:
# 성별에 따른 갯수
df1.loc[:,'성별'].value_counts()

암    545
수    508
거    260
Name: 성별, dtype: int64

## 연령

In [9]:
# 연령 2살~9살, 주로 2~5살의 말들이 있다.
# 연령별 카운트
df1.loc[:,'연령'].value_counts()

3    426
4    318
2    297
5    178
6     71
7     17
8      4
9      2
Name: 연령, dtype: int64

In [10]:
# 연령 비율
df1.loc[:,'연령'].value_counts(normalize=True)

3    0.324448
4    0.242193
2    0.226200
5    0.135567
6    0.054075
7    0.012947
8    0.003046
9    0.001523
Name: 연령, dtype: float64

## 조교사

In [11]:
# 조교사 컬럼에서 조교사이름과 번호 분리
# 조교사=Trainer
temp = df1.loc[:,"조교사"].apply(lambda x: re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", " ", x).split())
temp

0        [안해양, 26]
1        [리카디, 34]
2       [휴양마, 999]
3        [심승태, 37]
4         [홍대유, 6]
           ...    
1308      [강성오, 9]
1309     [안해양, 26]
1310     [안해양, 26]
1311     [이관호, 42]
1312      [우창구, 5]
Name: 조교사, Length: 1313, dtype: object

In [12]:
# 조교사 이름,번호 컬럽 추가
df1["조교사이름"] = temp.apply(lambda x: x[0])
df1["조교사번호"] = temp.apply(lambda x: int(x[1]))

# 조교사 컬럼 삭제
df1.drop("조교사", axis=1, inplace=True)
df1

,마번,마명,등급,성별,연령,레이팅1,전적,수득상금(천원),마주,부마명,모마명,조교사이름,조교사번호
0,45027,가나패션,국4,암,3,38,5(0/4/1),"40,800",김한수,올드패션드,찬란한여명,안해양,26
1,39021,가로수길,국2,수,6,73,13(5/3/0),"202,880",홍종철,채플로열,협동,리카디,34
2,43765,갈색향기,국5,암,3,26,10(0/2/1),"22,900",김광명,메니피,휴지호프,휴양마,999
3,44650,감곡선샤인,국5,암,3,30,6(1/0/0),"14,750",손칠규,에이스갤러퍼,드림갤러퍼,심승태,37
4,45187,강력한지금,국4,수,2,37,6(1/1/0),"27,550",금악목장,페더럴리스트,금악여제,홍대유,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,43514,히든삭스,외2,수,5,80,24(3/5/3),"238,700",김창식,RUN AWAY AND HIDE,DELIGHTFUL DAWN,강성오,9
1309,45908,히어로사랑,국6,암,2,0,1(0/1/0),"13,200",임재형,위드디스팅션,지봉사랑,안해양,26
1310,45380,히어로퀸,국6,암,2,0,1(0/0/0),0,임재형,선더모카신,라이브퀸,안해양,26
1311,42707,히트에이스,국5,암,4,28,12(0/2/1),"19,240",유수현,메니피,천년나래,이관호,42


## 전적

In [13]:
# 전적 컬럼 숫자 분리
temp = df1.loc[:,"전적"].apply(lambda x: re.sub(r"[^0-9]", " ", x).split())
temp

0        [5, 0, 4, 1]
1       [13, 5, 3, 0]
2       [10, 0, 2, 1]
3        [6, 1, 0, 0]
4        [6, 1, 1, 0]
            ...      
1308    [24, 3, 5, 3]
1309     [1, 0, 1, 0]
1310     [1, 0, 0, 0]
1311    [12, 0, 2, 1]
1312     [2, 1, 0, 0]
Name: 전적, Length: 1313, dtype: object

In [14]:
# 전적 컬럼 분할 후 새로운 컬럼으로 넣기
df1["총경기"] = temp.apply(lambda x: int(x[0]))
df1["1st"] = temp.apply(lambda x: int(x[1]))
df1["2st"] = temp.apply(lambda x: int(x[2]))
df1["3st"] = temp.apply(lambda x: int(x[3]))

# # 기존 전적 컬럼 삭제
df1.drop("전적", axis=1, inplace=True)
df1

,마번,마명,등급,성별,연령,레이팅1,수득상금(천원),마주,부마명,모마명,조교사이름,조교사번호,총경기,1st,2st,3st
0,45027,가나패션,국4,암,3,38,"40,800",김한수,올드패션드,찬란한여명,안해양,26,5,0,4,1
1,39021,가로수길,국2,수,6,73,"202,880",홍종철,채플로열,협동,리카디,34,13,5,3,0
2,43765,갈색향기,국5,암,3,26,"22,900",김광명,메니피,휴지호프,휴양마,999,10,0,2,1
3,44650,감곡선샤인,국5,암,3,30,"14,750",손칠규,에이스갤러퍼,드림갤러퍼,심승태,37,6,1,0,0
4,45187,강력한지금,국4,수,2,37,"27,550",금악목장,페더럴리스트,금악여제,홍대유,6,6,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,43514,히든삭스,외2,수,5,80,"238,700",김창식,RUN AWAY AND HIDE,DELIGHTFUL DAWN,강성오,9,24,3,5,3
1309,45908,히어로사랑,국6,암,2,0,"13,200",임재형,위드디스팅션,지봉사랑,안해양,26,1,0,1,0
1310,45380,히어로퀸,국6,암,2,0,0,임재형,선더모카신,라이브퀸,안해양,26,1,0,0,0
1311,42707,히트에이스,국5,암,4,28,"19,240",유수현,메니피,천년나래,이관호,42,12,0,2,1


## 성별 label encoding

In [15]:
# 라벨 인코더
# 2: 암, 1: 수, 0: 거
le = LabelEncoder()
df1["성별"] = le.fit_transform(df1['성별'])
df1

,마번,마명,등급,성별,연령,레이팅1,수득상금(천원),마주,부마명,모마명,조교사이름,조교사번호,총경기,1st,2st,3st
0,45027,가나패션,국4,2,3,38,"40,800",김한수,올드패션드,찬란한여명,안해양,26,5,0,4,1
1,39021,가로수길,국2,1,6,73,"202,880",홍종철,채플로열,협동,리카디,34,13,5,3,0
2,43765,갈색향기,국5,2,3,26,"22,900",김광명,메니피,휴지호프,휴양마,999,10,0,2,1
3,44650,감곡선샤인,국5,2,3,30,"14,750",손칠규,에이스갤러퍼,드림갤러퍼,심승태,37,6,1,0,0
4,45187,강력한지금,국4,1,2,37,"27,550",금악목장,페더럴리스트,금악여제,홍대유,6,6,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,43514,히든삭스,외2,1,5,80,"238,700",김창식,RUN AWAY AND HIDE,DELIGHTFUL DAWN,강성오,9,24,3,5,3
1309,45908,히어로사랑,국6,2,2,0,"13,200",임재형,위드디스팅션,지봉사랑,안해양,26,1,0,1,0
1310,45380,히어로퀸,국6,2,2,0,0,임재형,선더모카신,라이브퀸,안해양,26,1,0,0,0
1311,42707,히트에이스,국5,2,4,28,"19,240",유수현,메니피,천년나래,이관호,42,12,0,2,1


## 등급

등급을 산지와 level로 나누겠다  
산지: 국가(한국, 외국)  
level:1~6

In [16]:
# 등급 컬럼을 산지와 level로 분할
df1['산지'] = df1['등급'].apply(lambda x : x[0])
df1['level'] = df1['등급'].apply(lambda x : int(x[1]))

# 등급 칼럼 제거
df1.drop('등급', axis=1, inplace=True)
df1

,마번,마명,성별,연령,레이팅1,수득상금(천원),마주,부마명,모마명,조교사이름,조교사번호,총경기,1st,2st,3st,산지,level
0,45027,가나패션,2,3,38,"40,800",김한수,올드패션드,찬란한여명,안해양,26,5,0,4,1,국,4
1,39021,가로수길,1,6,73,"202,880",홍종철,채플로열,협동,리카디,34,13,5,3,0,국,2
2,43765,갈색향기,2,3,26,"22,900",김광명,메니피,휴지호프,휴양마,999,10,0,2,1,국,5
3,44650,감곡선샤인,2,3,30,"14,750",손칠규,에이스갤러퍼,드림갤러퍼,심승태,37,6,1,0,0,국,5
4,45187,강력한지금,1,2,37,"27,550",금악목장,페더럴리스트,금악여제,홍대유,6,6,1,1,0,국,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,43514,히든삭스,1,5,80,"238,700",김창식,RUN AWAY AND HIDE,DELIGHTFUL DAWN,강성오,9,24,3,5,3,외,2
1309,45908,히어로사랑,2,2,0,"13,200",임재형,위드디스팅션,지봉사랑,안해양,26,1,0,1,0,국,6
1310,45380,히어로퀸,2,2,0,0,임재형,선더모카신,라이브퀸,안해양,26,1,0,0,0,국,6
1311,42707,히트에이스,2,4,28,"19,240",유수현,메니피,천년나래,이관호,42,12,0,2,1,국,5


## 산지 label encoding

In [17]:
# 산지 컬럼
# 0: 국산말, 1: 외국말
df1['산지'] = pd.get_dummies(df1['산지'], drop_first=True)
df1

,마번,마명,성별,연령,레이팅1,수득상금(천원),마주,부마명,모마명,조교사이름,조교사번호,총경기,1st,2st,3st,산지,level
0,45027,가나패션,2,3,38,"40,800",김한수,올드패션드,찬란한여명,안해양,26,5,0,4,1,0,4
1,39021,가로수길,1,6,73,"202,880",홍종철,채플로열,협동,리카디,34,13,5,3,0,0,2
2,43765,갈색향기,2,3,26,"22,900",김광명,메니피,휴지호프,휴양마,999,10,0,2,1,0,5
3,44650,감곡선샤인,2,3,30,"14,750",손칠규,에이스갤러퍼,드림갤러퍼,심승태,37,6,1,0,0,0,5
4,45187,강력한지금,1,2,37,"27,550",금악목장,페더럴리스트,금악여제,홍대유,6,6,1,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,43514,히든삭스,1,5,80,"238,700",김창식,RUN AWAY AND HIDE,DELIGHTFUL DAWN,강성오,9,24,3,5,3,1,2
1309,45908,히어로사랑,2,2,0,"13,200",임재형,위드디스팅션,지봉사랑,안해양,26,1,0,1,0,0,6
1310,45380,히어로퀸,2,2,0,0,임재형,선더모카신,라이브퀸,안해양,26,1,0,0,0,0,6
1311,42707,히트에이스,2,4,28,"19,240",유수현,메니피,천년나래,이관호,42,12,0,2,1,0,5


In [18]:
df1.to_csv("data/경주마내역_전처리_1.csv", index=False, encoding='utf-8-sig')

In [19]:
df1

,마번,마명,성별,연령,레이팅1,수득상금(천원),마주,부마명,모마명,조교사이름,조교사번호,총경기,1st,2st,3st,산지,level
0,45027,가나패션,2,3,38,"40,800",김한수,올드패션드,찬란한여명,안해양,26,5,0,4,1,0,4
1,39021,가로수길,1,6,73,"202,880",홍종철,채플로열,협동,리카디,34,13,5,3,0,0,2
2,43765,갈색향기,2,3,26,"22,900",김광명,메니피,휴지호프,휴양마,999,10,0,2,1,0,5
3,44650,감곡선샤인,2,3,30,"14,750",손칠규,에이스갤러퍼,드림갤러퍼,심승태,37,6,1,0,0,0,5
4,45187,강력한지금,1,2,37,"27,550",금악목장,페더럴리스트,금악여제,홍대유,6,6,1,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,43514,히든삭스,1,5,80,"238,700",김창식,RUN AWAY AND HIDE,DELIGHTFUL DAWN,강성오,9,24,3,5,3,1,2
1309,45908,히어로사랑,2,2,0,"13,200",임재형,위드디스팅션,지봉사랑,안해양,26,1,0,1,0,0,6
1310,45380,히어로퀸,2,2,0,0,임재형,선더모카신,라이브퀸,안해양,26,1,0,0,0,0,6
1311,42707,히트에이스,2,4,28,"19,240",유수현,메니피,천년나래,이관호,42,12,0,2,1,0,5
